<a href="https://colab.research.google.com/github/viniciusrpb/cloud_image_classification/blob/main/phytoplankton_pretrainedmodels_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resultados Resnet50 - Phytoplankton

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install tensorflow_addons
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.7 MB/s eta 0:00:00


In [3]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense,GlobalAveragePooling2D ,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import SGD,Adam
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
import keras_tuner
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import os

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
def loadData():
    !cp -r "/content/drive/My Drive/plankton" "plankton"
    path_data = "plankton"

    list_subfolders = os.listdir(path_data)
    
    list_subfolders.sort()

    dataset_dict = {}

    dataset_dict['filename'] = []
    dataset_dict['label'] = []

    for folder in list_subfolders:

        list_images_path = os.listdir(path_data+"/"+folder)
        
        list_images_path.sort()

        for image_name in list_images_path:

            dataset_dict['filename'].append(folder+"/"+image_name)

            dataset_dict['label'].append(folder)
    
    df = pd.DataFrame.from_dict(dataset_dict)
    labels = list(df.columns)

    return df,labels

Hyperparameter optimization

In [5]:
def build_PreTrainedModel_resnet50(hp):

    num_labels=11

    model = Sequential()
    
    pre_trained_model = ResNet50(input_shape=(224,224,3),include_top=False,pooling ='avg',weights='imagenet', classes=num_labels)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    model.add(pre_trained_model)

    model.add(Flatten())
    model.add(Dense(hp.Choice('num_neurons',[32,64,128,256,1024,2048]),activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('prob_dropout',[0.1,0.2,0.3,0.4])))
    model.add(Dense(num_labels,activation='softmax'))

    f1_score = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

    adam = Adam(hp.Choice('learning_rate',[0.1,0.01,0.001,0.005,0.0001,0.0005,0.00001,0.00005,0.00002,0.0002]))
    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=[f1_score])
    
    return model

In [6]:
def build_PreTrainedModel_inceptionv3(hp):

    num_labels=11

    model = Sequential()
    
    pre_trained_model = InceptionV3(input_shape=(224,224,3),include_top=False,pooling ='avg',weights='imagenet', classes=num_labels)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    model.add(pre_trained_model)

    model.add(Flatten())
    model.add(Dense(hp.Choice('num_neurons',[32,64,128,256,1024,2048]),activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('prob_dropout',[0.1,0.2,0.3,0.4])))
    model.add(Dense(num_labels,activation='softmax'))

    f1_score = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

    adam = Adam(hp.Choice('learning_rate',[0.1,0.01,0.001,0.005,0.0001,0.0005,0.00001,0.00005,0.00002,0.0002]))
    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=[f1_score])
    
    return model

In [7]:
def build_PreTrainedModel_vgg16(hp):

    num_labels=11

    model = Sequential()
    
    pre_trained_model = VGG16(input_shape=(224,224,3),include_top=False,pooling ='avg',weights='imagenet', classes=num_labels)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    model.add(pre_trained_model)

    model.add(Flatten())
    model.add(Dense(hp.Choice('num_neurons',[32,64,128,256,1024,2048]),activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('prob_dropout',[0.1,0.2,0.3,0.4])))
    model.add(Dense(num_labels,activation='softmax'))

    f1_score = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

    adam = Adam(hp.Choice('learning_rate',[0.1,0.01,0.001,0.005,0.0001,0.0005,0.00001,0.00005,0.00002,0.0002]))
    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=[f1_score])
    
    return model

In [8]:
def getPreTrainedModel(units,prob,lr,num_labels,model_name):

    model = Sequential()
    
    if model_name == 'vgg16':
        pre_trained_model = VGG16(input_shape=(224,224,3),include_top=False,pooling ='avg',weights='imagenet', classes=num_labels)
    elif model_name == 'resnet50':
        pre_trained_model = ResNet50(input_shape=(224,224,3),include_top=False,pooling ='avg',weights='imagenet', classes=num_labels)
    else:
        pre_trained_model = InceptionV3(input_shape=(224,224,3),include_top=False,pooling ='avg',weights='imagenet', classes=num_labels)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    model.add(pre_trained_model)

    model.add(Flatten())
    model.add(Dense(units,activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(prob))
    model.add(Dense(num_labels,activation='softmax'))

    f1_score = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

    adam = Adam(learning_rate=lr)
    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=[f1_score])
    
    return model    

In [9]:
def hyperparameterOptimization(flag_da,num_labels,model,path_data):

    train, test = train_test_split(df, test_size=0.4, random_state=42)

    valid, test = train_test_split(test, test_size=0.5, random_state=42)

    agnostic_datagen = ImageDataGenerator(rescale=1./255)

    if flag_da == True:
        
        train_datagen = ImageDataGenerator(rescale=1./255,
                                           rotation_range=10,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True,
                                           brightness_range=[0.2,1.2])

        train_generator = train_datagen.flow_from_dataframe(train, directory = path_data,
                                                        x_col = "filename", y_col = "label",
                                                        batch_size=32,
                                                        seed=42,
                                                        class_mode = "categorical")
    else:
        train_generator = agnostic_datagen.flow_from_dataframe(train, directory = path_data,
                                                        x_col = "filename", y_col = "label",
                                                        batch_size=32,
                                                        seed=42,
                                                        class_mode = "categorical")   

    valid_generator = agnostic_datagen.flow_from_dataframe(valid, directory = path_data,
                                                        x_col = "filename", y_col = "label",
                                                        batch_size=32,
                                                        seed=42,
                                                        class_mode = "categorical")

    if model == 'resnet50':

        tuner = keras_tuner.RandomSearch(
            build_PreTrainedModel_resnet50,
            objective='val_loss',
            max_trials=5)
    elif model == 'vgg16':

        tuner = keras_tuner.RandomSearch(
            build_PreTrainedModel_vgg16,
            objective='val_loss',
            max_trials=5)
    else:

        tuner = keras_tuner.RandomSearch(
            build_PreTrainedModel_inceptionv3,
            objective='val_loss',
            max_trials=5)


    stop_early = EarlyStopping(monitor='val_loss', patience=5)

    tuner.search(train_generator, epochs=100, validation_data=valid_generator)

    best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]

    string = "The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is "+str(best_hps.get('num_neurons'))+", dropout is "+str(best_hps.get('prob_dropout'))+" and the optimal learning rate for the optimizer is "+str(best_hps.get('learning_rate'))+"."

    return best_hps,string

Stratified K-Fold Experiments

In [10]:
def kfoldExperiments(model,best_hps,flag_da,path_data,num_labels,epochs):

    skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

    X = np.array(df['filename'])
    y = np.array(df['label'])

    activation_f = 'softmax'
    lr = best_hps.get('learning_rate')
    prob = best_hps.get('prob_dropout')
    num_neurons = best_hps.get('num_neurons')

    f1_score = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

    test_f1_score = []
    test_precision_score = []
    test_recall_score = []

    matrices = []

    trial = 1

    predicted_targets = np.array([])
    actual_targets = np.array([])

    for train_ix, test_ix in skf.split(X,y):

        val_f1score = []
        val_loss = []
        
        train_list = []
        test_list = []

        for ind in train_ix:
            train_list.append([X[ind],y[ind]])
        
        for ind in test_ix:
            test_list.append([X[ind],y[ind]])

        X_train = pd.DataFrame(train_list, columns =['filename','label'])
        X_test = pd.DataFrame(test_list, columns =['filename','label'])

        agnostic_datagen = ImageDataGenerator(rescale=1./255)

        if flag_da == True:
        
            train_datagen = ImageDataGenerator(rescale=1./255,
                                            rotation_range=10,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            brightness_range=[0.2,1.2])

            train_generator = train_datagen.flow_from_dataframe(X_train, directory = path_data,
                                                            x_col = "filename", y_col = "label",
                                                            batch_size=32,
                                                            seed=42,
                                                            class_mode = "categorical")
        else:
            train_generator = agnostic_datagen.flow_from_dataframe(X_train, directory = path_data,
                                                            x_col = "filename", y_col = "label",
                                                            batch_size=32,
                                                            seed=42,
                                                            class_mode = "categorical")   

        test_generator = agnostic_datagen.flow_from_dataframe(X_test, directory = path_data,
                                                            x_col = "filename", y_col = "label",
                                                            batch_size=32,
                                                            seed=42,
                                                            class_mode = "categorical")
        
        early_stopping = EarlyStopping(monitor='loss', patience=5)

        model = getPreTrainedModel(num_neurons,prob,lr,num_labels,model_name)

        history_fine = model.fit(train_generator,
                        epochs=epochs,
                        batch_size=32,
                        callbacks=[early_stopping]
                        )
            
        f1 = history_fine.history['f1_score']

        loss = history_fine.history['loss']

        y_true = test_generator.labels
        y_prob = model.predict(test_generator)
        y_pred = np.argmax(y_prob,axis=1)

        test_recall_score.append(recall_score(y_true, y_pred, average='macro'))
        test_precision_score.append(recall_score(y_true, y_pred, average='macro'))
        test_f1_score.append(recall_score(y_true, y_pred, average='macro'))

        matrices.append(confusion_matrix(y_true, y_pred))

        predicted_targets = np.append(predicted_targets, y_pred)
        actual_targets = np.append(actual_targets, y_true)

        del model

    return test_f1_score,test_precision_score,test_recall_score,matrices,actual_targets,predicted_targets

In [11]:
def writeResults(experiment_name,string_besthps,test_f1_score,test_precision_score,test_recall_score,matrices):

    string = ''+string_besthps+'\n'

    string += 'f1_folds = ['
    for s in test_f1_score:
        string += str(s)+','

    string+= string[:-1]+']\n'

    string += 'precision_folds =['
    for s in test_precision_score:
        string += str(s)+','

    string+= string[:-1]+']\n'

    string += 'recall_folds = ['
    for s in test_recall_score:
        string += str(s)+','

    string+= string[:-1]+']\n'

    file_ans = open(experiment_name+'_metrics.txt','w')
    file_ans.write(string)
    file_ans.close()

In [12]:
def generate_confusion_matrix(cnf_matrix, classes, normalize=False, title='Confusion matrix'):
    if normalize:
        cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cnf_matrix, interpolation='nearest', cmap=plt.get_cmap('Blues'))
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cnf_matrix.max() / 2.

    for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
        plt.text(j, i, format(cnf_matrix[i, j], fmt), horizontalalignment="center",
                 color="white" if cnf_matrix[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return cnf_matrix

In [13]:
def plot_confusion_matrix(predicted_labels_list, y_test_list,name):
    cnf_matrix = confusion_matrix(y_test_list, predicted_labels_list)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    generate_confusion_matrix(cnf_matrix, classes=t, title='Confusion matrix, without normalization')
    plt.savefig(name+'_confmatrix'+'.png', bbox_inches='tight')

    # Plot normalized confusion matrix
    plt.figure()
    generate_confusion_matrix(cnf_matrix, classes=labels, normalize=True, title='Normalized confusion matrix')
    plt.savefig(name+'_normconfmatrix'+'.png', bbox_inches='tight')

Run All Experiments

In [ ]:
flag_da = [False,True]
models = ['proposed','resnet50','vgg16','inception_v3']

path_data = 'plankton'

for aug in flag_da:
    
    for model in models:
        if aug:
            experiment_name = path_data+'_'+model+'_aug_'
        else:
            experiment_name = path_data+'_'+model+'_'
            
        df,labels = loadData()

        num_labels = len(labels)

        best_hypers,string_hps = hyperparameterOptimization(flag_da,num_labels,model,path_data)

        f1s,precisions,recalls,matrices,y_true,y_pred = kfoldExperiments(model,best_hypers,path_data,flag_da,num_labels,25)

        writeResults(experiment_name,best_hypers,f1s,precisions,recalls,matrices)

        plot_confusion_matrix(y_pred, y_true,experiment_name)

Trial 4 Complete [00h 43m 37s]
val_loss: 0.22976617515087128

Best val_loss So Far: 0.2258644700050354
Total elapsed time: 03h 21m 40s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
32                |64                |num_neurons
0.1               |0.2               |prob_dropout
0.01              |0.0001            |learning_rate

Epoch 1/100
134/134 [==============================] - 32s 185ms/step - loss: 0.5332 - f1_score: 0.6016 - val_loss: 0.3954 - val_f1_score: 0.7128
Epoch 2/100
134/134 [==============================] - 23s 174ms/step - loss: 0.2490 - f1_score: 0.8397 - val_loss: 0.2438 - val_f1_score: 0.8105
Epoch 3/100
134/134 [==============================] - 23s 174ms/step - loss: 0.1762 - f1_score: 0.8895 - val_loss: 0.2981 - val_f1_score: 0.8256
Epoch 4/100
134/134 [==============================] - 27s 203ms/step - loss: 0.1479 - f1_score: 0.9297 - val_loss: 0.3066 - val_f1_score: 0.8166
Epoch 5/100
134/134 [==========================